In [20]:
import os
from dotenv import load_dotenv
from dune_client.client import DuneClient
import pandas as pd

load_dotenv()
dune = DuneClient(os.getenv('DUNE_API_KEY'))

def unpack(query_result):
    if query_result and query_result.result and query_result.result.rows:
        df = pd.DataFrame(query_result.result.rows)
    else:
        print("No data available in the query result.")
        df = pd.DataFrame()  # Create an empty DataFrame if no data is available
    return df

corn_df = unpack(dune.get_latest_result(4014290))
corn_df.to_csv('Corn Trades.csv')
wallet_analysis = corn_df['wallet_address'].value_counts().reset_index()
wallet_analysis.columns = ['wallet_address', 'transaction_count']


,wallet_address,transaction_count
0,<a href=https://polygonscan.com//address/0x28e...,17
1,<a href=https://polygonscan.com//address/0xb4f...,17
2,<a href=https://polygonscan.com//address/0x068...,12
3,<a href=https://polygonscan.com//address/0xa5e...,10
4,<a href=https://polygonscan.com//address/0x55a...,9
5,<a href=https://polygonscan.com//address/0x6e7...,4
6,<a href=https://polygonscan.com//address/0xeea...,3
7,<a href=https://polygonscan.com//address/0x19e...,3
8,<a href=https://polygonscan.com//address/0x407...,3
9,<a href=https://polygonscan.com//address/0x1f1...,3


In [19]:
poster_df = unpack(dune.get_latest_result(4013378))

poster_df

,address,net_posters,posters_bought,posters_sold,total_transactions,unique_posters_bought,unique_posters_sold
0,0x39da41747a83aee658334415666f3ef92dd0d541,0,1243,1243,2486,1031,1031
1,0xca86d57519dbfe34a25eef0923b259ab07986b71,0,1092,1092,2184,389,389
2,0x995bf64daba77afc7a26cb2eb8ddf1404921904a,0,552,552,1104,322,322
3,0xfcb11f4f6749dd517aaa9f38035fecc1fd91291d,1,533,532,1065,518,517
4,0xa580bfa51f601abaf1ac4d95c4979a9a8490503a,519,519,0,519,519,0
...,...,...,...,...,...,...,...
6030,0x9bbcfb97747884e1c1c804a3f537b4b28c3039ef,1,1,0,1,1,0
6031,0x00c56b48db3a47b87b4a1377dcc7bea024025645,1,1,0,1,1,0
6032,0x3a60eca4bbbd3248b5dc11d233eeed2115946c2f,1,1,0,1,1,0
6033,0x24584d75176cdf226db337514b71afede7ae1f72,1,1,0,1,1,0


In [26]:
import os
import json
import asyncio
from ring_doorbell import Ring, Auth
from oauthlib.oauth2 import MissingTokenError

cache_file = "token.cache"

def token_updated(token):
    with open(cache_file, "w") as f:
        f.write(json.dumps(token))

def otp_callback():
    auth_code = input("2FA code: ")
    return auth_code

async def main():
    if os.path.isfile(cache_file):
        with open(cache_file) as f:
            token = json.load(f)
    else:
        token = None

    auth = Auth("MyProject/1.0", token, token_updated)

    try:
        await asyncio.to_thread(auth.fetch_token, os.getenv('RING_USERNAME'), os.getenv('RING_PASSWORD'))
    except MissingTokenError:
        await asyncio.to_thread(auth.fetch_token, os.getenv('RING_USERNAME'), os.getenv('RING_PASSWORD'), otp_callback())

    ring = Ring(auth)
    await ring.update_data()

    devices = ring.devices()
    for device in devices['doorbots']:
        print('Doorbot name: {}'.format(device.name))
        print('Battery: {}'.format(device.battery_life))
        print('Wifi: {}'.format(device.wifi_signal_strength))

def run_main():
    asyncio.run(main())

if __name__ == "__main__":
    try:
        run_main()
    except RuntimeError as e:
        if str(e) == "asyncio.run() cannot be called from a running event loop":
            loop = asyncio.get_running_loop()
            loop.create_task(main())
        else:
            raise

/var/folders/xm/n7pv6dns0t3fsf_v922_qf3h0000gn/T/ipykernel_25452/3118239688.py:49: RuntimeWarning: coroutine 'main' was never awaited
  loop.create_task(main())


2024-08-24 12:05:29,368 ERROR asyncio Task exception was never retrieved
future: <Task finished name='Task-10' coro=<main() done, defined at /var/folders/xm/n7pv6dns0t3fsf_v922_qf3h0000gn/T/ipykernel_25452/3118239688.py:17> exception=Requires2FAError()>
Traceback (most recent call last):
  File "/Users/pragma/miniforge3/envs/based/lib/python3.10/site-packages/ring_doorbell/auth.py", line 107, in async_fetch_token
    self._token = self._oauth_client.parse_request_body_response(
  File "/Users/pragma/miniforge3/envs/based/lib/python3.10/site-packages/oauthlib/oauth2/rfc6749/clients/base.py", line 427, in parse_request_body_response
    self.token = parse_token_response(body, scope=scope)
  File "/Users/pragma/miniforge3/envs/based/lib/python3.10/site-packages/oauthlib/oauth2/rfc6749/parameters.py", line 441, in parse_token_response
    validate_token_parameters(params)
  File "/Users/pragma/miniforge3/envs/based/lib/python3.10/site-packages/oauthlib/oauth2/rfc6749/parameters.py", line 4